In [1]:
import warnings
warnings.filterwarnings("ignore")


In [ ]:
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten, Activation,  MaxPooling2D, BatchNormalization
from keras import regularizers
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
# loading the dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# # building the input vector from the 32x32 pixels
X_train = X_train.reshape(X_train.shape[0], 32, 32, 3)
X_test = X_test.reshape(X_test.shape[0], 32, 32, 3)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# normalizing the data to help with the training
X_train /= 255
X_test /= 255

# one-hot encoding using keras' numpy-related utilities
n_classes = 10
print("Shape before one-hot encoding: ", y_train.shape)
Y_train = np_utils.to_categorical(y_train, n_classes)
Y_test = np_utils.to_categorical(y_test, n_classes)
print("Shape after one-hot encoding: ", Y_train.shape)

# building a linear stack of layers with the sequential model
model = Sequential()
L2_coeff = 0.0005
# convolutional layer
model.add(Conv2D(100, (3, 3), padding='same', activation='relu', input_shape=[32,32,3],kernel_regularizer=regularizers.l2(L2_coeff)))
model.add(BatchNormalization())
# convolutional layer
model.add(Conv2D(100, kernel_size=(3,3), padding='same', activation='relu',kernel_regularizer=regularizers.l2(L2_coeff)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(50, kernel_size=(3,3), padding='same', activation='relu',kernel_regularizer=regularizers.l2(L2_coeff)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))

# flatten output of conv
model.add(Flatten())

# hidden layer
model.add(Dense(500, activation='relu',kernel_regularizer=regularizers.l2(L2_coeff)))
model.add(BatchNormalization())
model.add(Dense(250, activation='relu',kernel_regularizer=regularizers.l2(L2_coeff)))
model.add(BatchNormalization())
# output layer
model.add(Dense(10, activation='softmax'))

# compiling the sequential model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

# train with augmented data to prevent overfitting
datagen = ImageDataGenerator(
            rotation_range=25,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.15,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.15,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=True)  # randomly flip images
datagen.fit(X_train)
model.fit(datagen.flow(X_train, Y_train, batch_size=500), steps_per_epoch=100, epochs=50, validation_data=(X_test, Y_test))

In [8]:
# train with original data for one epoch
model.fit(X_train, Y_train, batch_size=500, epochs=1, validation_data=(X_test, Y_test))

100/100 [==============================] - 6s 62ms/step - loss: 0.3232 - accuracy: 0.9804 - val_loss: 0.9349 - val_accuracy: 0.8090


In [9]:
model.save('regular_cnn.h5')
